# BÁO CÁO TIẾN ĐỘ GIỮA KỲ
Lớp TTNT - 147729, Nhóm G05 

## 1. Phương pháp và thuật toán sử dụng
### Tính toán tương đồng: 
- Dữ liệu các bài hát được biểu diễn thành các vector sử dụng phương pháp TF-IDF (Term Frequency-Inverse Document Frequency). Tính toán ma trận tương đồng cosine giữa các vector biểu diễn của các bài hát. Ma trận này chứa thông tin về mức độ tương đồng giữa mỗi cặp bài hát trong tập dữ liệu.
### Gợi ý âm nhạc: 
- Khi người dùng chọn một bài hát từ giao diện, hệ thống sẽ tìm kiếm index của bài hát đó trong dataframe.
- Sau đó, hệ thống sẽ sắp xếp các bài hát khác dựa trên mức độ tương đồng với bài hát đã chọn. Mức độ tương đồng cao hơn sẽ đưa các bài hát đó lên phía trên trong danh sách gợi ý.
- Bốn bước tiếp theo sau khi tìm kiếm index của bài hát đó trong dataframe là để lấy thông tin về nghệ sĩ và ảnh bìa album của các bài hát được gợi ý, thông qua API của Spotify. 

### Tải dữ liệu và tiền xử lý: 
- Chuỗi văn bản trong cột 'text' được chuyển thành chữ thường để đảm bảo tính nhất quán, các ký tự khôgn phải là chữ cái hoặc số và các ký tự xuống dòng được loại bỏ để làm sạch dữ liệu
### Tách từ (Tokenization) và Stemming:
- Sử dụng thư viện NLTK (Natural Language Toolkit) để tách chuỗi văn bản thành các từ riêng lẻ (Tokenization).
- Sau đó, các từ này được chuyển về dạng gốc (Stemming) bằng cách sử dụng thuật toán PorterStemmer trong NLTK.
- Quá trình này giúp làm giảm kích thước của từ vựng và chuẩn hóa các từ, từ đó cải thiện khả năng hiểu và xử lý của mô hình.

### Tính toán độ tương đồng: 
- Sử dụng cosine_similarity từ scikit-learn để tính toán độ tương đồng cosine giữa các vector biểu diễn văn bản.
Độ tương đồng cosine đo lường sự tương đồng giữa các mục văn bản dựa trên nội dung của chúng.






## 2. Kết quả thực hiện giưã kỳ 

### data_processing


In [ ]:
import pandas as pd
#Loading data
df = pd.read_csv("spotify_millsongdata.csv")
df.shape
#Kiểm tra các ô null
df.isnull().sum()
df = df.drop('link', axis = 1)
df.head(5)
df['text'][0]
df.sample(5000)
df = df.sample(5000)
df.head(5)
Cleaning
df['text']=df['text'].str.lower().replace('r^\w\s',' ').replace(r'\n',' ', regex=True)
df.head(5)
df['text'][50238]
Tokenization và Vectorization
import nltk
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
def token(txt):
    token = nltk.word_tokenize(txt) # tách chuỗi văn bản thành các từ  riêng lẻ
    a = [stemmer.stem(w) for w in token] # duyệt qua từng từ trong token và chuyển về dạng gốc
    return " ".join(a) # ghép các từ trong list a thành một chuỗi, ngăn cách bởi dấu " "
token("you are beautiful")
df['text'].apply(lambda x: token(x))
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
tfid = TfidfVectorizer(analyzer='word', stop_words='english')
# sử dụng từ riêng lẻ trong văn bản để tạo vector, các option khác: char, char_wb
# stopwords: loại bỏ các từ không mang ý nghĩa như "a", "an", "the", "me", ...
matrix = tfid.fit_transform(df['text'])
similer = cosine_similarity(matrix)
## Kết quả là một ma trận đối xứng, giá trị ở hàng thứ i và cột thứ j thể hiện độ tương đồng cosin giữa hai vector biểu diễn văn bản i và j trong tập dữ liệu.
similer[0]


### recommender_system

In [ ]:
import pickle
import streamlit as st
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

ClientID = "" #"70a9fb89662f4dac8d07321b259eaad7"
ClientSecret = "" #"4d6710460d764fbbb8d8753dc094d131"

# Initialize the Spotify client
client_credentials_manager = SpotifyClientCredentials(client_id = ClientID, client_secret = ClientSecret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

def get_song_album_cover_url(song_name, artist_name):
    search_query = f"track:{song_name} artist:{artist_name}"
    results = sp.search(q = search_query, type = "track")

    if results and results["tracks"]["items"]:
        track = results["tracks"]["items"][0]
        album_cover_url = track["album"]["images"][0]["url"]
        print(album_cover_url)
        return album_cover_url
    else:
        return "https://i.postimg.cc/0QNxYz4V/social.png"

def recommend(song):
    index = music[music['song'] == song].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse = True, key = lambda x: x[1])
    recommended_music_names = []
    recommended_music_posters = []
    for i in distances[1:6]:
        # fetch the movie poster
        artist = music.iloc[i[0]].artist
        print(artist)
        print(music.iloc[i[0]].song)
        recommended_music_posters.append(get_song_album_cover_url(music.iloc[i[0]].song, artist))
        recommended_music_names.append(music.iloc[i[0]].song)

    return recommended_music_names,recommended_music_posters

st.header('Music Recommender System')
music = pickle.load(open('df.pkl','rb'))
similarity = pickle.load(open('similarity.pkl','rb'))

music_list = music['song'].values
selected_movie = st.selectbox(
    "Type or select a song from the dropdown",
    music_list
)

if st.button('Show Recommendation'):
    recommended_music_names,recommended_music_posters = recommend(selected_movie)
    col1, col2, col3, col4, col5= st.columns(5)
    with col1:
        st.text(recommended_music_names[0])
        st.image(recommended_music_posters[0])
    with col2:
        st.text(recommended_music_names[1])
        st.image(recommended_music_posters[1])

    with col3:
        st.text(recommended_music_names[2])
        st.image(recommended_music_posters[2])
    with col4:
        st.text(recommended_music_names[3])
        st.image(recommended_music_posters[3])
    with col5:
        st.text(recommended_music_names[4])
        st.image(recommended_music_posters[4])

### Demo chương trình
![text](demo_1.png)


## 3. Khó khăn cần tiếp tục giải quyết
### Xử lý dữ liệu không chuẩn:
- Dữ liệu có thể chứa các ký tự đặc biệt, dấu câu hoặc ký tự không mong muốn khác. Cần phải xử lý dữ liệu để loại bỏ những ký tự này trước khi áp dụng các phương pháp tiền xử lý dữ liệu như chuyển thành chữ thường, loại bỏ stop words, tokenization và stemming.
### Hiệu suất tính toán:
- Các phương pháp tiền xử lý dữ liệu và tính toán độ tương đồng có thể tốn nhiều thời gian và tài nguyên tính toán, đặc biệt là với các tập dữ liệu lớn. Cần tối ưu hóa mã để cải thiện hiệu suất tính toán.
### Xử lý lỗi và ngoại lệ:
- Cần xử lý các trường hợp lỗi và ngoại lệ một cách chính xác, bao gồm việc xử lý dữ liệu không đầy đủ hoặc không hợp lệ từ API Spotify, lỗi trong quá trình tính toán, hoặc lỗi khi tải dữ liệu.
### Tối ưu hóa gợi ý:
- Có thể cần phải cải thiện thuật toán gợi ý để đảm bảo rằng các bài hát được gợi ý là chất lượng và phù hợp với người dùng hơn. Điều này có thể đòi hỏi sự phức tạp hóa hoặc mở rộng thuật toán gợi ý hiện tại.